In [1]:
import sys
import torch
import numpy as np
sys.path.append('..')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf
from erank.regularization import EffectiveRankRegularization
import matplotlib.pyplot as plt

/home/max/anaconda3/envs/erank/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def erank_np(matrix_A: np.ndarray):
    s = np.linalg.svd(matrix_A, compute_uv=False)
    probs = s / s.sum()
    entropy = (- probs * np.log(probs)).sum()
    return np.exp(entropy)

In [3]:
def erank_torch(matrix_A: torch.Tensor, center_matrix_A: bool=False) -> torch.Tensor:
    """Calculates the effective rank of a matrix.
    Args:
        matrix_A (torch.Tensor): Matrix of shape m x n. 
        center_matrix_A (bool): Center the matrix 
    Returns:
        torch.Tensor: Effective rank of matrix_A
    """
    assert matrix_A.ndim == 2
    _, s, _ = torch.pca_lowrank(matrix_A, center=center_matrix_A, niter=1, q=min(matrix_A.shape[0], matrix_A.shape[1]))
    # normalizes input s -> scale independent!
    return torch.exp(torch.distributions.Categorical(s).entropy())

In [4]:
matrix_A = np.random.normal(size=(2,3))
v = np.random.normal(size=(1,3))
matrix_A

array([[ 0.01901721, -0.68034222,  0.68561504],
       [-0.06375954, -0.68777692, -0.1437013 ]])

In [5]:
erank_np(matrix_A), erank_torch(torch.from_numpy(matrix_A))

(1.8908603346885038, tensor(1.8909, dtype=torch.float64))

In [6]:
cov_A = np.cov(matrix_A)
erank_np(cov_A), np.linalg.matrix_rank(cov_A)

In [16]:
matrix_A_concat_zero = np.concatenate([np.zeros((1,matrix_A.shape[1])), matrix_A])
matrix_A_concat_zero

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.01901721, -0.68034222,  0.68561504],
       [-0.06375954, -0.68777692, -0.1437013 ]])

In [17]:
erank_np(matrix_A_concat_zero)

1.890860334688504

In [18]:
erank_torch(torch.from_numpy(matrix_A_concat_zero))

tensor(1.8909, dtype=torch.float64)